A notebook to test Pytorch.

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import numpy as np
import os
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib
import matplotlib.pyplot as plt

# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 24
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [7]:
torch.zeros(3, device=torch.device('cpu'))

tensor([0., 0., 0.])

In [11]:
w = torch.tensor([3,5,4.0])
w.dot(torch.ones(3))

tensor(12.)

## Torch 2d to 1d and vice versa

In [223]:
torch.eye(3).matmul(torch.tensor([[2., 3, 1]]).T)

tensor([[2.],
        [3.],
        [1.]])

In [222]:
torch.tensor([[2., 3, 1]]).T

tensor([[2.],
        [3.],
        [1.]])

In [30]:
V = torch.randn(3, 1)
V

tensor([[0.9760],
        [0.6724],
        [1.2658]])

In [31]:
V.reshape(-1)

tensor([0.9760, 0.6724, 1.2658])

In [34]:
v = torch.rand(4)
v

tensor([0.5324, 0.6466, 0.7210, 0.6114])

In [35]:
v.reshape(4, 1)

tensor([[0.5324],
        [0.6466],
        [0.7210],
        [0.6114]])

## Torch distributions
https://pytorch.org/docs/stable/distributions.html

In [12]:
import torch.distributions as dists

In [24]:
dis = dists.Normal(0, 1)
dis.log_prob(torch.ones(5))

tensor([-1.4189, -1.4189, -1.4189, -1.4189, -1.4189])

### Grad of log (Normal density)

$$ \nabla_x \log p(x) = -x$$ where $p(x) = \mathcal{N}(0,1)$.

Input is a scalar

In [109]:
x = torch.tensor([3.0], requires_grad=True)
y = 2*x
lp = dis.log_prob(y)

In [102]:
lp.backward(retain_graph=True) 
x.grad

tensor([-12.])

In [104]:
torch.autograd.grad([lp], [x, y], retain_graph=True, only_inputs=True)

(tensor([-12.]), tensor([-6.]))

Input is n x 1

In [186]:
X = torch.tensor([[-3, 1.0, 2]], requires_grad=True).T
Lp = dis.log_prob(X)
sum_lp = torch.sum(Lp)

In [187]:
torch.autograd.grad(sum_lp, X, retain_graph=True, only_inputs=True)

(tensor([[ 3.],
         [-1.],
         [-2.]]),)

## Multivariate normal

Assume $p(x) = \mathcal{N}(x \mid \mu, I)$. Then

$$ \nabla_x \log p(x) = -(x-\mu) $$

In [212]:
mdist = dists.MultivariateNormal(
    loc=torch.tensor([1, 2]), covariance_matrix=torch.eye(2))

X = torch.tensor([
    [0, 0.0],
    [0, 1.0],
    [1.0, 4],
])
X.requires_grad = True
logp = mdist.log_prob(X)
torch.autograd.grad(torch.sum(logp), X, retain_graph=True, only_inputs=True)[0]

tensor([[ 1.,  2.],
        [ 1.,  1.],
        [ 0., -2.]])